<a href="https://colab.research.google.com/github/Nding-Capstone/BLSTM/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1MB 27kB/s 
     |████████████████████████████████| 8.8MB 31.3MB/s 
  Found existing installation: torch 1.5.0+cu101
    Uninstalling torch-1.5.0+cu101:
      Successfully uninstalled torch-1.5.0+cu101
  Found existing installation: torchvision 0.6.0+cu101
    Uninstalling torchvision-0.6.0+cu101:
      Successfully uninstalled torchvision-0.6.0+cu101


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks/')

In [0]:
import os
import json
import numpy as np
import torch 
import torch.nn as nn

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [0]:
train_total_class = np.load('/content/drive/My Drive/Colab Notebooks/캡스톤_동영상/캡스톤_동영상/npy/train_total.npy')
test_total_class = np.load('/content/drive/My Drive/Colab Notebooks/캡스톤_동영상/캡스톤_동영상/npy/test_total.npy')

In [0]:
print(train_total_class.shape)
print(test_total_class.shape)

(150, 40, 8)
(150, 20, 8)


In [0]:
class_cnt = 20

In [0]:
x_data = torch.from_numpy(train_total_class[:,:class_cnt*2,:])
x_test_data = torch.from_numpy(test_total_class[:,:class_cnt,:])
tmp_train = list()
tmp_test = list()
for i in range(class_cnt) :  
  tmp_train.append(i)
  tmp_train.append(i)
  tmp_test.append(i)
y_test_data = np.array(tmp_test)
y_test_data = torch.from_numpy(y_test_data)
y_data = np.array(tmp_train)
y_data = torch.from_numpy(y_data)

In [0]:
print(x_data.size())
print(y_data.size())
print(x_test_data.size())
print(y_test_data.size())

torch.Size([150, 40, 8])
torch.Size([40])
torch.Size([150, 20, 8])
torch.Size([20])


In [0]:
class LSTM(nn.Module):
 
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
 
        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True, bidirectional=True)
 
        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim*2, output_dim)
 
    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers*2, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers*2, self.batch_size, self.hidden_dim))
 
    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        input = input.float()
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        feature_map = (lstm_out[-1].view(self.batch_size, -1))
        feature_map = feature_map.view(feature_map.size(0),feature_map.size(1),1,1)
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))

        return y_pred,feature_map

In [0]:
num_batch_size = 1
torch.manual_seed(123)
torch.cuda.manual_seed(123)
base_model = LSTM(8, 32, batch_size=num_batch_size, output_dim=class_cnt, num_layers=2)

In [0]:
from torch.utils.data import  TensorDataset, DataLoader

dataset = TensorDataset(x_data.permute(1,0,2), y_data.type(torch.LongTensor))
loader = DataLoader(dataset, batch_size=num_batch_size, shuffle=True)

dataset_test = TensorDataset(x_test_data.permute(1,0,2), y_test_data.type(torch.LongTensor))
loader_test = DataLoader(dataset_test, batch_size=num_batch_size, shuffle=True)

optimiser = torch.optim.Adam(base_model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()
hist = np.zeros(100)

#####################
# Train model
#####################
  
best_acc_test = 0
best_acc_train = 0

for t in range(100):
  
    batch_loss = 0.0
    acc_val = 0.0
    acc_val_train = 0.0
    sum_test = 0
    sum_train = 0
    correct = 0
    correct_train = 0

    for xx, yy in loader:
      sum_train += 1
      # Initialise hidden state
      # Don't do this if you want your LSTM to be stateful
      base_model.hidden = base_model.init_hidden()

      xx = xx.permute(1,0,2)
      # Clear stored gradient
      base_model.zero_grad()

      # Forward pass
      y_pred, _ = base_model(xx)
      train_label = torch.argmax(y_pred)
      if train_label.item() == yy.item() : 
        correct_train += 1
      loss = loss_fn(y_pred, yy)
      batch_loss += loss.item()

      # Zero out gradient, else they will accumulate between epochs
      optimiser.zero_grad()

      # Backward pass
      loss.backward()

      # Update parameters
      optimiser.step()
    
    for xx_test, yy_test in loader_test:
      sum_test += 1
      xx_test = xx_test.permute(1,0,2)
      y_test_pred, _ = base_model(xx_test)
      _,test_label = torch.topk(y_test_pred,3)
      if yy_test.item() in test_label : 
        correct += 1
    
    
    acc_val_train = correct_train/sum_train
    acc_val = correct/sum_test

    print("Epoch : ", t ,"Loss : ", batch_loss, "Train_ACC : " , acc_val_train , "Test_ACC : " , acc_val)
    if best_acc_test <= acc_val and best_acc_train <= acc_val_train:
      best_acc_test = acc_val
      best_acc_train = acc_val_train
      torch.save(base_model.state_dict(),'/content/drive/My Drive/Colab Notebooks/chekcpoint_20200506_best_top3.pth')
      print("save_check_point")


print("Best_top3_check_point - Train_ACC : ", best_acc_train , " Test_ACC : ", best_acc_test)

Epoch :  0 Loss :  121.73178839683533 Train_ACC :  0.025 Test_ACC :  0.15
save_check_point
Epoch :  1 Loss :  123.0019690990448 Train_ACC :  0.0 Test_ACC :  0.2
Epoch :  2 Loss :  120.89031386375427 Train_ACC :  0.05 Test_ACC :  0.2
save_check_point
Epoch :  3 Loss :  119.83864688873291 Train_ACC :  0.05 Test_ACC :  0.25
save_check_point
Epoch :  4 Loss :  117.54682850837708 Train_ACC :  0.075 Test_ACC :  0.3
save_check_point
Epoch :  5 Loss :  117.67785322666168 Train_ACC :  0.05 Test_ACC :  0.3
Epoch :  6 Loss :  112.58218804001808 Train_ACC :  0.1 Test_ACC :  0.15
Epoch :  7 Loss :  109.82833695411682 Train_ACC :  0.125 Test_ACC :  0.3
save_check_point
Epoch :  8 Loss :  111.43946471810341 Train_ACC :  0.15 Test_ACC :  0.25
Epoch :  9 Loss :  104.83003604412079 Train_ACC :  0.175 Test_ACC :  0.25
Epoch :  10 Loss :  101.36404103040695 Train_ACC :  0.275 Test_ACC :  0.25
Epoch :  11 Loss :  100.84474366903305 Train_ACC :  0.225 Test_ACC :  0.3
save_check_point
Epoch :  12 Loss :  92.

In [0]:
from netvlad import NetVLAD
from netvlad import EmbedNet
import sklearn.metrics.pairwise as pairwise

In [0]:
# Adapted from: https://omoindrot.github.io/triplet-loss

##############
# CAUTION !! #
##############
# At the moment both batch_all and batch_hard seem to pass the error testing from the source
# mentioned above, but in practice for my experiments batch_hard converged to margin and
# batch_all the error exploded. So for me both are not functional at the moment, but 
# _get_triplet_mask can be used to get all valid triplets and then F.triplet_margin_loss()
# can be used to get them running. See resnet.py for how the losses are implemented in this
# project.

import torch
from torch.autograd import Variable

def _pairwise_distances(embeddings, squared=False):
    """Compute the 2D matrix of distances between all the embeddings.
    Args:
        embeddings: Variable of shape (batch_size, embed_dim)
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.
    Returns:
        pairwise_distances: Variable of shape (batch_size, batch_size)
    """

    # Get the dot product between all embeddings
    # shape (batch_size, batch_size)
    dot_product = torch.mm(embeddings, embeddings.t())

    # Get squared L2 norm for each embedding. We can just take the diagonal of `dot_product`.
    # This also provides more numerical stability (the diagonal of the result will be exactly 0).
    # shape (batch_size,)
    square_norm = torch.diag(dot_product)

    # Compute the pairwise distance matrix as we have:
    # ||a - b||^2 = ||a||^2  - 2 <a, b> + ||b||^2
    # shape (batch_size, batch_size)
    distances = torch.unsqueeze(square_norm, 0) - 2.0 * dot_product + torch.unsqueeze(square_norm, 1)

    # Because of computation errors, some distances might be negative so we put everything >= 0.0
    distances = torch.clamp(distances, min=0.0)

    if not squared:
        # Not sure if needed for pytorch but does not harm
        # Because the gradient of sqrt is infinite when distances == 0.0 (ex: on the diagonal)
        # we need to add a small epsilon where distances == 0.0
        mask = torch.eq(distances, 0.0).float()
        distances = distances + mask * 1e-16

        distances = torch.sqrt(distances)

        # Correct the epsilon added: set the distances on the mask to be exactly 0.0
        distances = distances * (1.0 - mask)

    return distances

def _get_anchor_positive_triplet_mask(labels):
    """Return a 2D mask where mask[a, p] is True iff a and p are distinct and have same label.
    Args:
        labels: torch.Tensor with shape [batch_size]
    Returns:
        mask: Varieble with torch.ByteTensor with shape [batch_size, batch_size]
    """
    # Check that i and j are distinct
    indices_equal = torch.eye(labels.size(0)).bool()
    if labels.is_cuda:
        indices_equal = indices_equal.cuda()
    indices_not_equal = ~indices_equal

    # Check if labels[i] == labels[j]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = torch.eq(torch.unsqueeze(labels, 0), torch.unsqueeze(labels, 1))

    # Combine the two masks
    mask = indices_not_equal & labels_equal

    return mask


def _get_anchor_negative_triplet_mask(labels):
    """Return a 2D mask where mask[a, n] is True iff a and n have distinct labels.
    Args:
        labels: torch.Tensor with shape [batch_size]
    Returns:
        mask: Variable with torch.ByteTensor with shape [batch_size, batch_size]
    """
    # Check if labels[i] != labels[k]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = torch.eq(torch.unsqueeze(labels, 0), torch.unsqueeze(labels, 1))

    mask = ~labels_equal

    return mask


def _get_triplet_mask(labels):
    """Return a 3D mask where mask[a, p, n] is True iff the triplet (a, p, n) is valid.
    A triplet (i, j, k) is valid if:
        - i, j, k are distinct
        - labels[i] == labels[j] and labels[i] != labels[k]
    Args:
        labels: torch.Tensor with shape [batch_size]
    """
    # Check that i, j and k are distinct
    indices_equal = torch.eye(labels.size(0)).byte()
    if labels.is_cuda:
        indices_equal = indices_equal.cuda()
    indices_not_equal = ~indices_equal
    i_not_equal_j = torch.unsqueeze(indices_not_equal, 2)
    i_not_equal_k = torch.unsqueeze(indices_not_equal, 1)
    j_not_equal_k = torch.unsqueeze(indices_not_equal, 0)

    distinct_indices = (i_not_equal_j & i_not_equal_k) & j_not_equal_k


    # Check if labels[i] == labels[j] and labels[i] != labels[k]
    label_equal = torch.eq(torch.unsqueeze(labels, 0), torch.unsqueeze(labels, 1))
    #if labels.is_cuda:
    #    label_equal = label_equal.cuda()
    i_equal_j = torch.unsqueeze(label_equal, 2)
    i_equal_k = torch.unsqueeze(label_equal, 1)

    valid_labels = i_equal_j & (~i_equal_k)
    
    # Combine the two masks
    mask = distinct_indices & valid_labels.type(torch.uint8)

    return mask

def batch_all_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.
    We generate all the valid triplets and average the loss over the positive ones.
    Args:
        labels: Variable with labels of the batch, of size (batch_size,)
        embeddings: Variable with tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.
    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = _pairwise_distances(embeddings, squared=squared)

    anchor_positive_dist = torch.unsqueeze(pairwise_dist, 2)
    anchor_negative_dist = torch.unsqueeze(pairwise_dist, 1)

    # Compute a 3D tensor of size (batch_size, batch_size, batch_size)
    # triplet_loss[i, j, k] will contain the triplet loss of anchor=i, positive=j, negative=k
    # Uses broadcasting where the 1st argument has shape (batch_size, batch_size, 1)
    # and the 2nd (batch_size, 1, batch_size)
    triplet_loss = anchor_positive_dist - anchor_negative_dist + margin

    # Put to zero the invalid triplets
    # (where label(a) != label(p) or label(n) == label(a) or a == p)
    mask = _get_triplet_mask(labels)
    mask = Variable(mask.float())
    triplet_loss = mask * triplet_loss

    # Remove negative losses (i.e. the easy triplets)
    triplet_loss = torch.clamp(triplet_loss, min=0.0)

    # Count number of positive triplets (where triplet_loss > 0)
    valid_triplets = torch.gt(triplet_loss, 1e-16)
    num_positive_triplets = valid_triplets.sum().float()
    num_valid_triplets = mask.sum()
    fraction_positive_triplets = num_positive_triplets / (num_valid_triplets + 1e-16)

    # Get final mean triplet loss over the positive valid triplets
    triplet_loss = triplet_loss.sum() / (num_positive_triplets + 1e-16)

    return triplet_loss, fraction_positive_triplets

def batch_hard_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.
    For each anchor, we get the hardest positive and hardest negative to form a triplet.
    Args:
        labels: Variable with labels of the batch, of size (batch_size,)
        embeddings: Variable with tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.
    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = _pairwise_distances(embeddings, squared=squared)

    # For each anchor, get the hardest positive
    # First, we need to get a mask for every valid positive (they should have same label)
    mask_anchor_positive = _get_anchor_positive_triplet_mask(labels)
    mask_anchor_positive = Variable(mask_anchor_positive.float())

    # We put to 0 any element where (a, p) is not valid (valid if a != p and label(a) == label(p))
    anchor_positive_dist = mask_anchor_positive * pairwise_dist

    # shape (batch_size, 1)
    hardest_positive_dist, _ = torch.max(anchor_positive_dist, dim=1, keepdim=True)

    # For each anchor, get the hardest negative
    # First, we need to get a mask for every valid negative (they should have different labels)
    mask_anchor_negative = _get_anchor_negative_triplet_mask(labels)
    mask_anchor_negative = Variable(mask_anchor_negative.float())

    # We add the maximum value in each row to the invalid negatives (label(a) == label(n))
    max_anchor_negative_dist, _ = torch.max(pairwise_dist, dim=1, keepdim=True)
    anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - mask_anchor_negative)

    # shape (batch_size,)
    hardest_negative_dist, _ = torch.min(anchor_negative_dist, dim=1, keepdim=True)

    # Combine biggest d(a, p) and smallest d(a, n) into final triplet loss
    triplet_loss = torch.clamp(hardest_positive_dist - hardest_negative_dist + margin, min=0.0)

    # Get final mean triplet loss
    triplet_loss = triplet_loss.mean()

    return triplet_loss

In [0]:
def largest_indices(array: np.ndarray, n: int) -> tuple:
    """Returns the n largest indices from a numpy array.
    Arguments:
        array {np.ndarray} -- data array
        n {int} -- number of elements to select
    Returns:
        tuple[np.ndarray, np.ndarray] -- tuple of ndarray
        each ndarray is index
    """
    flat = array.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    return ((indices+1)/2+0.5).astype('int64')

In [0]:
dataset_VLAD = TensorDataset(x_data.permute(1,0,2), y_data.type(torch.LongTensor))
loader_VLAD = DataLoader(dataset_VLAD, batch_size=10, shuffle=True)

dataset_VLAD_test = TensorDataset(x_test_data.permute(1,0,2), y_test_data.type(torch.LongTensor))
loader_VLAD_test = DataLoader(dataset_VLAD_test, batch_size=10, shuffle=True)

In [0]:
back_bone_model = LSTM(8, 32, batch_size=10, output_dim=class_cnt, num_layers=2)
back_bone_model.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/chekcpoint_20200506_best_top3.pth'), strict=True)

<All keys matched successfully>

In [0]:
# Define model for embedding
net_vlad = NetVLAD(num_clusters=20, dim=64, alpha=1.0)
model = EmbedNet(back_bone_model, net_vlad).cuda()
# criterion = HardTripletLoss(margin=0.1).cuda()
# criterion = batch_all_triplet_loss(margin=0.1).cuda()
optimiser_netvlad = torch.optim.Adam(model.parameters(), lr=0.001)

In [0]:
for t in range(3000):

  correct = 0
  
  for xx_vlad, yy_vlad in loader_VLAD:
    xx_vlad = xx_vlad.permute(1,0,2)
    output = model(xx_vlad.cuda())
    triplet_loss = batch_all_triplet_loss(yy_vlad.cuda(),output,margin=0.1)
    # Zero out gradient, else they will accumulate between epochs
    optimiser_netvlad.zero_grad()

    # Backward pass
    triplet_loss[0].backward()

    # Update parameters
    optimiser_netvlad.step()

  for xx_vlad_test, yy_vlad_test in loader_VLAD_test:
    xx_vlad_test = xx_vlad_test.permute(1,0,2)
    vald_out_test = model(xx_vlad_test.cuda())

  similarity = pairwise.cosine_similarity(X=vald_out_test.cpu().detach().numpy(), Y=vald_out_train.cpu().detach().numpy(), dense_output=True)
  
  for i in range(len(similarity)) :
    if yy_vlad_test[i].item() in largest_indices(similarity[i],3) : 
      correct += 1
  
  acc_val = correct/10

  if t % 10 == 0:
    print("Epoch ", t, "Triplet_loss: ", triplet_loss[0].item(), "Test_ACC :", acc_val)

Epoch  0 Triplet_loss:  0.11106157302856445 Test_ACC : 0.1
Epoch  10 Triplet_loss:  0.10560329258441925 Test_ACC : 0.2
Epoch  20 Triplet_loss:  0.07312006503343582 Test_ACC : 0.1
Epoch  30 Triplet_loss:  0.09156176447868347 Test_ACC : 0.0
Epoch  40 Triplet_loss:  0.10847645998001099 Test_ACC : 0.1
Epoch  50 Triplet_loss:  0.09567942470312119 Test_ACC : 0.1
Epoch  60 Triplet_loss:  0.0 Test_ACC : 0.1
Epoch  70 Triplet_loss:  0.0673879086971283 Test_ACC : 0.0
Epoch  80 Triplet_loss:  0.0 Test_ACC : 0.0
Epoch  90 Triplet_loss:  0.023257054388523102 Test_ACC : 0.0
Epoch  100 Triplet_loss:  0.0 Test_ACC : 0.1
Epoch  110 Triplet_loss:  0.0018243566155433655 Test_ACC : 0.1
Epoch  120 Triplet_loss:  0.0 Test_ACC : 0.1
Epoch  130 Triplet_loss:  0.0 Test_ACC : 0.1
Epoch  140 Triplet_loss:  0.10040651261806488 Test_ACC : 0.2
Epoch  150 Triplet_loss:  0.0528729110956192 Test_ACC : 0.2
Epoch  160 Triplet_loss:  0.0 Test_ACC : 0.1
Epoch  170 Triplet_loss:  0.10126084834337234 Test_ACC : 0.0
Epoch  1

KeyboardInterrupt: ignored

In [0]:
vald_out_train.shape

torch.Size([40, 1280])

In [0]:
torch.save(model.state_dict(),'/content/drive/My Drive/Colab Notebooks/VLAD_Checkpoint_20200506.pth')

In [0]:
dataset_VLAD_test = TensorDataset(x_test_data.permute(1,0,2), y_test_data.type(torch.LongTensor))
loader_VLAD_test = DataLoader(dataset_VLAD_test, batch_size=10, shuffle=True)

In [0]:
back_bone_model = LSTM(8, 32, batch_size=10, output_dim=class_cnt, num_layers=2)
net_vlad = NetVLAD(num_clusters=20, dim=64, alpha=1.0)
model = EmbedNet(back_bone_model, net_vlad).cuda()
model.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/VLAD_Checkpoint_20200506.pth'), strict=True)

<All keys matched successfully>

In [0]:
model.eval()

for xx_vlad_test, yy_vlad_test in loader_VLAD_test:
  xx_vlad_test = xx_vlad_test.permute(1,0,2)
  vald_out_test = model(xx_vlad_test.cuda())

In [0]:
vald_out_test.size()

torch.Size([10, 1280])

In [0]:
similarity = pairwise.cosine_similarity(X=vald_out_test.cpu().detach().numpy(), Y=vald_out_train.cpu().detach().numpy(), dense_output=True)
#similarity = pairwise.cosine_similarity(X=vald_out_train.cpu().detach().numpy(), Y=vald_out_train.cpu().detach().numpy(), dense_output=True)

In [0]:
similarity

array([[0.5478743 , 0.53230685, 0.4983186 , 0.48963085, 0.8509887 ,
        0.6484924 , 0.6814134 , 0.45905724, 0.4808666 , 0.5824408 ,
        0.5939707 , 0.31204283, 0.5617833 , 0.45794606, 0.4652688 ,
        0.59405875, 0.601035  , 0.5306729 , 0.44606566, 0.49725428,
        0.68929625, 0.49754393, 0.39432213, 0.5127539 , 0.39100105,
        0.6662994 , 0.6197829 , 0.36044106, 0.5415908 , 0.5732816 ,
        0.3393296 , 0.6062944 , 0.3307428 , 0.5208565 , 0.59452766,
        0.6707636 , 0.69013876, 0.56984925, 0.5065799 , 0.7351494 ],
       [0.69541454, 0.760062  , 0.5185545 , 0.697226  , 0.45075762,
        0.47041717, 0.47507495, 0.5299672 , 0.93823665, 0.44241655,
        0.58940727, 0.5294286 , 0.65996844, 0.42700607, 0.56729025,
        0.66303515, 0.40691844, 0.4702618 , 0.6711115 , 0.90476036,
        0.45361468, 0.4212109 , 0.61699426, 0.5325421 , 0.73800075,
        0.50529695, 0.77864844, 0.69477594, 0.44714504, 0.6533525 ,
        0.4814344 , 0.4333452 , 0.53020203, 0.4

In [0]:
def largest_indices(array: np.ndarray, n: int) -> tuple:
    """Returns the n largest indices from a numpy array.
    Arguments:
        array {np.ndarray} -- data array
        n {int} -- number of elements to select
    Returns:
        tuple[np.ndarray, np.ndarray] -- tuple of ndarray
        each ndarray is index
    """
    flat = array.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    return ((indices+1)/2+0.5).astype('int64')

In [0]:
for i in range(len(similarity)) :
  print(largest_indices(similarity[i],3))

[ 3 20 19]
[ 5 10 14]
[ 6 17 13]
[14 13  6]
[19 12  4]
[ 8 20  5]
[19  3 18]
[10 12 17]
[13  4  7]
[ 5 10  1]


In [0]:
yy_vlad_test

tensor([ 5,  1, 16, 15, 14, 12, 10,  2,  7, 13])